In [1]:
from sklearn.ensemble import RandomForestClassifier
import deepchem as dc
import numpy as np
import pandas as pd
import tempfile
import chemprop
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import roc_auc_score
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras import backend as K
from tensorflow.keras import initializers
from tensorflow.keras import layers
from tensorflow.keras import regularizers
from sklearn.linear_model import LogisticRegression
import xgboost
import os
from sklearn.svm import SVC

2022-11-07 12:54:05.280598: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-07 12:54:05.389247: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-11-07 12:54:05.389263: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-11-07 12:54:05.411061: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-11-07 12:54:05.941228: W tensorflow/stream_executor/platform/de

In [2]:
AUC_ROC=[]
model=[]
def list_auc(list_f,list_m):
    main_all_feature=pd.DataFrame()
    for feature in list_f:
        for m in list_m:
            read_main=pd.read_csv('pred_'+m+'_main_set_'+feature+'.csv')
            df_main=pd.DataFrame(read_main)
            pred1=[]
            for p in df_main['pred']:
                if p<=0.5:
                    p=0
                else:
                    p=1
                pred1.append(p)
            main_all_feature.insert(loc=len(main_all_feature.columns),column=m+feature,value=pred1)


    external_all_feature=pd.DataFrame()
    for feature in list_f:
        for m in list_m:
            read_main=pd.read_csv('pred_'+m+'_external_set_'+feature+'.csv')
            df_main=pd.DataFrame(read_main)
            pred1=[]
            for p in df_main['pred']:
                if p<=0.5:
                    p=0
                else:
                    p=1
                pred1.append(p)
            external_all_feature.insert(loc=len(external_all_feature.columns),column=m+feature,value=pred1)
    
    read_m = pd.read_csv("main_set_plus_AtomPairFP.csv")
    df_m = pd.DataFrame(read_m)
    a1 = df_m['active']
    c1=[]
    for b1 in a1:
        if b1==1:
            b1=0
        else:
            b1=1
        c1.append(b1)

    y_m=pd.DataFrame()
    y_m.insert(loc=0, column='active', value=a1)
    y_m.insert(loc=1, column='inactive', value=c1)
    X_m=main_all_feature

    read_e = pd.read_csv("external_set_plus_AtomPairFP.csv")
    df_e = pd.DataFrame(read_e)
    a2 = df_e['active']
    c1=[]
    for b1 in a2:
        if b1==1:
            b1=0
        else:
            b1=1
        c1.append(b1)

    y_e=pd.DataFrame()
    y_e.insert(loc=0, column='active', value=a2)
    y_e.insert(loc=1, column='inactive', value=c1)
    X_e=external_all_feature

    batch_size = 8192
    num_classes = 2
    model = Sequential()
    model.add(Dense(800,
                    activation='relu',
                    input_dim=len(X_m.columns),
                    kernel_initializer = initializers.RandomNormal(stddev=0.02),
                    bias_initializer = initializers.RandomNormal(mean=1.),
                    kernel_regularizer = regularizers.l2(1e-1)))
    model.add(Dropout(0.5))
    model.add(Dense(2, activation='softmax'))
    opt = keras.optimizers.Adam(learning_rate=0.001)

    model.compile(loss=keras.losses.categorical_crossentropy,
                  optimizer=opt,
                  metrics=['AUC'])
    model.fit(X_m, y_m,
              batch_size=batch_size,
              epochs=100)
    score = model.evaluate(X_e, y_e, verbose=0)
    print(score[1])
    AUC_ROC.append(score[1])
    model=''

In [3]:
m='all_model'
a=['AtomPairFP','Charge',
       'Kappa','MAP4','MOE','MorganFP','Topology']
b=['xgb','fcnn','lr','dmpnn']
list_auc(a,b)
model.append(m)

Epoch 1/100


2022-11-07 12:54:10.243682: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-07 12:54:10.244217: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcufft.so.10'; dlerror: libcufft.so.10: cannot open shared object file: No such file or directory
2022-11-07 12:54:10.244265: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcurand.so.10'; dlerror: libcurand.so.10: cannot open shared object file: No such file or directory
2022-11-07 12:54:10.244305: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcusolver.so.11'; dlerror: libcusolver.so.11: cannot open shared object file: No such file or directory
2022-11-07 12:54:10.244345: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could no

1/1 [==============================] - 0s 403ms/step - loss: 2.1050 - auc: 0.7844
Epoch 2/100
1/1 [==============================] - 0s 28ms/step - loss: 1.7380 - auc: 0.7722
Epoch 3/100
1/1 [==============================] - 0s 30ms/step - loss: 1.5948 - auc: 0.7179
Epoch 4/100
1/1 [==============================] - 0s 29ms/step - loss: 1.5797 - auc: 0.6678
Epoch 5/100
1/1 [==============================] - 0s 33ms/step - loss: 1.4187 - auc: 0.7554
Epoch 6/100
1/1 [==============================] - 0s 30ms/step - loss: 1.2716 - auc: 0.8378
Epoch 7/100
1/1 [==============================] - 0s 34ms/step - loss: 1.2058 - auc: 0.8735
Epoch 8/100
1/1 [==============================] - 0s 34ms/step - loss: 1.1613 - auc: 0.8888
Epoch 9/100
1/1 [==============================] - 0s 34ms/step - loss: 1.0873 - auc: 0.9106
Epoch 10/100
1/1 [==============================] - 0s 35ms/step - loss: 0.9862 - auc: 0.9287
Epoch 11/100
1/1 [==============================] - 0s 33ms/step - loss: 0.9134 

1/1 [==============================] - 0s 30ms/step - loss: 0.2369 - auc: 0.9901
Epoch 89/100
1/1 [==============================] - 0s 32ms/step - loss: 0.2322 - auc: 0.9919
Epoch 90/100
1/1 [==============================] - 0s 25ms/step - loss: 0.2248 - auc: 0.9917
Epoch 91/100
1/1 [==============================] - 0s 29ms/step - loss: 0.2380 - auc: 0.9897
Epoch 92/100
1/1 [==============================] - 0s 28ms/step - loss: 0.2254 - auc: 0.9926
Epoch 93/100
1/1 [==============================] - 0s 24ms/step - loss: 0.2240 - auc: 0.9925
Epoch 94/100
1/1 [==============================] - 0s 24ms/step - loss: 0.2258 - auc: 0.9924
Epoch 95/100
1/1 [==============================] - 0s 25ms/step - loss: 0.2355 - auc: 0.9901
Epoch 96/100
1/1 [==============================] - 0s 27ms/step - loss: 0.2356 - auc: 0.9909
Epoch 97/100
1/1 [==============================] - 0s 31ms/step - loss: 0.2254 - auc: 0.9922
Epoch 98/100
1/1 [==============================] - 0s 27ms/step - loss: 

In [4]:
m='remove_AtomPairFP'
a=['Charge',
       'Kappa','MAP4','MOE','MorganFP','Topology']
b=['xgb','fcnn','lr','dmpnn']
list_auc(a,b)
model.append(m)

Epoch 1/100
1/1 [==============================] - 0s 343ms/step - loss: 2.8432 - auc: 0.2358
Epoch 2/100
1/1 [==============================] - 0s 32ms/step - loss: 1.6700 - auc: 0.5908
Epoch 3/100
1/1 [==============================] - 0s 36ms/step - loss: 1.4132 - auc: 0.8147
Epoch 4/100
1/1 [==============================] - 0s 26ms/step - loss: 1.5255 - auc: 0.8421
Epoch 5/100
1/1 [==============================] - 0s 36ms/step - loss: 1.6431 - auc: 0.8467
Epoch 6/100
1/1 [==============================] - 0s 35ms/step - loss: 1.6910 - auc: 0.8511
Epoch 7/100
1/1 [==============================] - 0s 27ms/step - loss: 1.6568 - auc: 0.8596
Epoch 8/100
1/1 [==============================] - 0s 26ms/step - loss: 1.5783 - auc: 0.8673
Epoch 9/100
1/1 [==============================] - 0s 31ms/step - loss: 1.4600 - auc: 0.8766
Epoch 10/100
1/1 [==============================] - 0s 28ms/step - loss: 1.3150 - auc: 0.8848
Epoch 11/100
1/1 [==============================] - 0s 25ms/step - l

1/1 [==============================] - 0s 29ms/step - loss: 0.2895 - auc: 0.9866
Epoch 89/100
1/1 [==============================] - 0s 27ms/step - loss: 0.2945 - auc: 0.9854
Epoch 90/100
1/1 [==============================] - 0s 24ms/step - loss: 0.3069 - auc: 0.9823
Epoch 91/100
1/1 [==============================] - 0s 22ms/step - loss: 0.2974 - auc: 0.9849
Epoch 92/100
1/1 [==============================] - 0s 25ms/step - loss: 0.2931 - auc: 0.9849
Epoch 93/100
1/1 [==============================] - 0s 24ms/step - loss: 0.2904 - auc: 0.9858
Epoch 94/100
1/1 [==============================] - 0s 28ms/step - loss: 0.2943 - auc: 0.9840
Epoch 95/100
1/1 [==============================] - 0s 30ms/step - loss: 0.2949 - auc: 0.9843
Epoch 96/100
1/1 [==============================] - 0s 26ms/step - loss: 0.2862 - auc: 0.9861
Epoch 97/100
1/1 [==============================] - 0s 24ms/step - loss: 0.2834 - auc: 0.9869
Epoch 98/100
1/1 [==============================] - 0s 29ms/step - loss: 

In [5]:
m='remove_Charge'
a=['AtomPairFP',
       'Kappa','MAP4','MOE','MorganFP','Topology']
b=['xgb','fcnn','lr','dmpnn']
list_auc(a,b)
model.append(m)

Epoch 1/100
1/1 [==============================] - 0s 342ms/step - loss: 1.5546 - auc: 0.7357
Epoch 2/100
1/1 [==============================] - 0s 30ms/step - loss: 1.5916 - auc: 0.8204
Epoch 3/100
1/1 [==============================] - 0s 30ms/step - loss: 1.3899 - auc: 0.8390
Epoch 4/100
1/1 [==============================] - 0s 31ms/step - loss: 1.2647 - auc: 0.8349
Epoch 5/100
1/1 [==============================] - 0s 33ms/step - loss: 1.1830 - auc: 0.8376
Epoch 6/100
1/1 [==============================] - 0s 30ms/step - loss: 1.1352 - auc: 0.8473
Epoch 7/100
1/1 [==============================] - 0s 37ms/step - loss: 0.9934 - auc: 0.9083
Epoch 8/100
1/1 [==============================] - 0s 35ms/step - loss: 0.9593 - auc: 0.9187
Epoch 9/100
1/1 [==============================] - 0s 32ms/step - loss: 0.8898 - auc: 0.9408
Epoch 10/100
1/1 [==============================] - 0s 28ms/step - loss: 0.8741 - auc: 0.9440
Epoch 11/100
1/1 [==============================] - 0s 39ms/step - l

1/1 [==============================] - 0s 29ms/step - loss: 0.2308 - auc: 0.9916
Epoch 89/100
1/1 [==============================] - 0s 27ms/step - loss: 0.2326 - auc: 0.9906
Epoch 90/100
1/1 [==============================] - 0s 35ms/step - loss: 0.2321 - auc: 0.9912
Epoch 91/100
1/1 [==============================] - 0s 29ms/step - loss: 0.2353 - auc: 0.9901
Epoch 92/100
1/1 [==============================] - 0s 29ms/step - loss: 0.2221 - auc: 0.9930
Epoch 93/100
1/1 [==============================] - 0s 27ms/step - loss: 0.2248 - auc: 0.9922
Epoch 94/100
1/1 [==============================] - 0s 29ms/step - loss: 0.2365 - auc: 0.9904
Epoch 95/100
1/1 [==============================] - 0s 26ms/step - loss: 0.2395 - auc: 0.9895
Epoch 96/100
1/1 [==============================] - 0s 25ms/step - loss: 0.2335 - auc: 0.9908
Epoch 97/100
1/1 [==============================] - 0s 27ms/step - loss: 0.2287 - auc: 0.9913
Epoch 98/100
1/1 [==============================] - 0s 23ms/step - loss: 

In [6]:
m='remove_Kappa'
a=['AtomPairFP',
       'Charge','MAP4','MOE','MorganFP','Topology']
b=['xgb','fcnn','lr','dmpnn']
list_auc(a,b)
model.append(m)

Epoch 1/100
1/1 [==============================] - 0s 344ms/step - loss: 2.0216 - auc: 0.4356
Epoch 2/100
1/1 [==============================] - 0s 26ms/step - loss: 1.4663 - auc: 0.7844
Epoch 3/100
1/1 [==============================] - 0s 34ms/step - loss: 1.5018 - auc: 0.8368
Epoch 4/100
1/1 [==============================] - 0s 30ms/step - loss: 1.5334 - auc: 0.8562
Epoch 5/100
1/1 [==============================] - 0s 27ms/step - loss: 1.5080 - auc: 0.8665
Epoch 6/100
1/1 [==============================] - 0s 31ms/step - loss: 1.4408 - auc: 0.8714
Epoch 7/100
1/1 [==============================] - 0s 29ms/step - loss: 1.2738 - auc: 0.8866
Epoch 8/100
1/1 [==============================] - 0s 29ms/step - loss: 1.1440 - auc: 0.8973
Epoch 9/100
1/1 [==============================] - 0s 37ms/step - loss: 1.0531 - auc: 0.9031
Epoch 10/100
1/1 [==============================] - 0s 37ms/step - loss: 0.9977 - auc: 0.9068
Epoch 11/100
1/1 [==============================] - 0s 33ms/step - l

1/1 [==============================] - 0s 27ms/step - loss: 0.2312 - auc: 0.9922
Epoch 89/100
1/1 [==============================] - 0s 26ms/step - loss: 0.2408 - auc: 0.9905
Epoch 90/100
1/1 [==============================] - 0s 22ms/step - loss: 0.2374 - auc: 0.9913
Epoch 91/100
1/1 [==============================] - 0s 29ms/step - loss: 0.2410 - auc: 0.9906
Epoch 92/100
1/1 [==============================] - 0s 27ms/step - loss: 0.2449 - auc: 0.9893
Epoch 93/100
1/1 [==============================] - 0s 25ms/step - loss: 0.2414 - auc: 0.9904
Epoch 94/100
1/1 [==============================] - 0s 27ms/step - loss: 0.2396 - auc: 0.9899
Epoch 95/100
1/1 [==============================] - 0s 31ms/step - loss: 0.2389 - auc: 0.9912
Epoch 96/100
1/1 [==============================] - 0s 25ms/step - loss: 0.2399 - auc: 0.9909
Epoch 97/100
1/1 [==============================] - 0s 24ms/step - loss: 0.2351 - auc: 0.9914
Epoch 98/100
1/1 [==============================] - 0s 26ms/step - loss: 

In [7]:
m='remove_MAP4'
a=['AtomPairFP',
       'Charge','Kappa','MOE','MorganFP','Topology']
b=['xgb','fcnn','lr','dmpnn']
list_auc(a,b)
model.append(m)

Epoch 1/100
1/1 [==============================] - 0s 341ms/step - loss: 1.5619 - auc: 0.7137
Epoch 2/100
1/1 [==============================] - 0s 29ms/step - loss: 1.6024 - auc: 0.8071
Epoch 3/100
1/1 [==============================] - 0s 33ms/step - loss: 1.4268 - auc: 0.8173
Epoch 4/100
1/1 [==============================] - 0s 25ms/step - loss: 1.3113 - auc: 0.8023
Epoch 5/100
1/1 [==============================] - 0s 42ms/step - loss: 1.2602 - auc: 0.7808
Epoch 6/100
1/1 [==============================] - 0s 35ms/step - loss: 1.1902 - auc: 0.8004
Epoch 7/100
1/1 [==============================] - 0s 28ms/step - loss: 1.0924 - auc: 0.8515
Epoch 8/100
1/1 [==============================] - 0s 29ms/step - loss: 1.0143 - auc: 0.8882
Epoch 9/100
1/1 [==============================] - 0s 26ms/step - loss: 0.9804 - auc: 0.9018
Epoch 10/100
1/1 [==============================] - 0s 31ms/step - loss: 0.9397 - auc: 0.9131
Epoch 11/100
1/1 [==============================] - 0s 44ms/step - l

1/1 [==============================] - 0s 25ms/step - loss: 0.2765 - auc: 0.9853
Epoch 89/100
1/1 [==============================] - 0s 23ms/step - loss: 0.2820 - auc: 0.9848
Epoch 90/100
1/1 [==============================] - 0s 26ms/step - loss: 0.2822 - auc: 0.9845
Epoch 91/100
1/1 [==============================] - 0s 26ms/step - loss: 0.2719 - auc: 0.9873
Epoch 92/100
1/1 [==============================] - 0s 26ms/step - loss: 0.2703 - auc: 0.9872
Epoch 93/100
1/1 [==============================] - 0s 26ms/step - loss: 0.2834 - auc: 0.9841
Epoch 94/100
1/1 [==============================] - 0s 27ms/step - loss: 0.2822 - auc: 0.9845
Epoch 95/100
1/1 [==============================] - 0s 36ms/step - loss: 0.2782 - auc: 0.9854
Epoch 96/100
1/1 [==============================] - 0s 26ms/step - loss: 0.2847 - auc: 0.9838
Epoch 97/100
1/1 [==============================] - 0s 32ms/step - loss: 0.2752 - auc: 0.9863
Epoch 98/100
1/1 [==============================] - 0s 27ms/step - loss: 

In [8]:
m='remove_MOE'
a=['AtomPairFP',
       'Charge','Kappa','MAP4','MorganFP','Topology']
b=['xgb','fcnn','lr','dmpnn']
list_auc(a,b)
model.append(m)

Epoch 1/100
1/1 [==============================] - 0s 346ms/step - loss: 1.7127 - auc: 0.5794
Epoch 2/100
1/1 [==============================] - 0s 32ms/step - loss: 1.5295 - auc: 0.7900
Epoch 3/100
1/1 [==============================] - 0s 27ms/step - loss: 1.5481 - auc: 0.8253
Epoch 4/100
1/1 [==============================] - 0s 31ms/step - loss: 1.5024 - auc: 0.8327
Epoch 5/100
1/1 [==============================] - 0s 32ms/step - loss: 1.3658 - auc: 0.8443
Epoch 6/100
1/1 [==============================] - 0s 31ms/step - loss: 1.2125 - auc: 0.8579
Epoch 7/100
1/1 [==============================] - 0s 28ms/step - loss: 1.1445 - auc: 0.8506
Epoch 8/100
1/1 [==============================] - 0s 26ms/step - loss: 1.1214 - auc: 0.8378
Epoch 9/100
1/1 [==============================] - 0s 39ms/step - loss: 1.0790 - auc: 0.8461
Epoch 10/100
1/1 [==============================] - 0s 33ms/step - loss: 0.9991 - auc: 0.8784
Epoch 11/100
1/1 [==============================] - 0s 31ms/step - l

1/1 [==============================] - 0s 24ms/step - loss: 0.2557 - auc: 0.9887
Epoch 89/100
1/1 [==============================] - 0s 30ms/step - loss: 0.2464 - auc: 0.9908
Epoch 90/100
1/1 [==============================] - 0s 23ms/step - loss: 0.2523 - auc: 0.9894
Epoch 91/100
1/1 [==============================] - 0s 26ms/step - loss: 0.2611 - auc: 0.9882
Epoch 92/100
1/1 [==============================] - 0s 24ms/step - loss: 0.2539 - auc: 0.9898
Epoch 93/100
1/1 [==============================] - 0s 24ms/step - loss: 0.2541 - auc: 0.9892
Epoch 94/100
1/1 [==============================] - 0s 27ms/step - loss: 0.2494 - auc: 0.9906
Epoch 95/100
1/1 [==============================] - 0s 26ms/step - loss: 0.2523 - auc: 0.9900
Epoch 96/100
1/1 [==============================] - 0s 23ms/step - loss: 0.2488 - auc: 0.9904
Epoch 97/100
1/1 [==============================] - 0s 28ms/step - loss: 0.2522 - auc: 0.9892
Epoch 98/100
1/1 [==============================] - 0s 24ms/step - loss: 

In [9]:
m='remove_MorganFP'
a=['AtomPairFP',
       'Charge','Kappa','MAP4','MOE','Topology']
b=['xgb','fcnn','lr','dmpnn']
list_auc(a,b)
model.append(m)

Epoch 1/100
1/1 [==============================] - 0s 355ms/step - loss: 2.0748 - auc: 0.3849
Epoch 2/100
1/1 [==============================] - 0s 31ms/step - loss: 1.4542 - auc: 0.7653
Epoch 3/100
1/1 [==============================] - 0s 32ms/step - loss: 1.5270 - auc: 0.8138
Epoch 4/100
1/1 [==============================] - 0s 28ms/step - loss: 1.5745 - auc: 0.8402
Epoch 5/100
1/1 [==============================] - 0s 35ms/step - loss: 1.5717 - auc: 0.8508
Epoch 6/100
1/1 [==============================] - 0s 25ms/step - loss: 1.4985 - auc: 0.8562
Epoch 7/100
1/1 [==============================] - 0s 27ms/step - loss: 1.3636 - auc: 0.8669
Epoch 8/100
1/1 [==============================] - 0s 29ms/step - loss: 1.2721 - auc: 0.8605
Epoch 9/100
1/1 [==============================] - 0s 32ms/step - loss: 1.1307 - auc: 0.8726
Epoch 10/100
1/1 [==============================] - 0s 36ms/step - loss: 1.0669 - auc: 0.8691
Epoch 11/100
1/1 [==============================] - 0s 48ms/step - l

1/1 [==============================] - 0s 33ms/step - loss: 0.2678 - auc: 0.9890
Epoch 89/100
1/1 [==============================] - 0s 39ms/step - loss: 0.2741 - auc: 0.9876
Epoch 90/100
1/1 [==============================] - 0s 27ms/step - loss: 0.2719 - auc: 0.9877
Epoch 91/100
1/1 [==============================] - 0s 28ms/step - loss: 0.2685 - auc: 0.9883
Epoch 92/100
1/1 [==============================] - 0s 26ms/step - loss: 0.2755 - auc: 0.9860
Epoch 93/100
1/1 [==============================] - 0s 27ms/step - loss: 0.2675 - auc: 0.9888
Epoch 94/100
1/1 [==============================] - 0s 27ms/step - loss: 0.2680 - auc: 0.9881
Epoch 95/100
1/1 [==============================] - 0s 23ms/step - loss: 0.2683 - auc: 0.9885
Epoch 96/100
1/1 [==============================] - 0s 26ms/step - loss: 0.2706 - auc: 0.9875
Epoch 97/100
1/1 [==============================] - 0s 26ms/step - loss: 0.2637 - auc: 0.9886
Epoch 98/100
1/1 [==============================] - 0s 27ms/step - loss: 

In [10]:
m='remove_Topology'
a=['AtomPairFP',
       'Charge','Kappa','MAP4','MOE','MorganFP']
b=['xgb','fcnn','lr','dmpnn']
list_auc(a,b)
model.append(m)

Epoch 1/100
1/1 [==============================] - 0s 344ms/step - loss: 1.7593 - auc: 0.5450
Epoch 2/100
1/1 [==============================] - 0s 29ms/step - loss: 1.4826 - auc: 0.7992
Epoch 3/100
1/1 [==============================] - 0s 31ms/step - loss: 1.5061 - auc: 0.8347
Epoch 4/100
1/1 [==============================] - 0s 33ms/step - loss: 1.4593 - auc: 0.8492
Epoch 5/100
1/1 [==============================] - 0s 30ms/step - loss: 1.3336 - auc: 0.8623
Epoch 6/100
1/1 [==============================] - 0s 30ms/step - loss: 1.2121 - auc: 0.8705
Epoch 7/100
1/1 [==============================] - 0s 26ms/step - loss: 1.1149 - auc: 0.8750
Epoch 8/100
1/1 [==============================] - 0s 32ms/step - loss: 1.0805 - auc: 0.8671
Epoch 9/100
1/1 [==============================] - 0s 30ms/step - loss: 1.0461 - auc: 0.8657
Epoch 10/100
1/1 [==============================] - 0s 28ms/step - loss: 1.0135 - auc: 0.8740
Epoch 11/100
1/1 [==============================] - 0s 27ms/step - l

1/1 [==============================] - 0s 27ms/step - loss: 0.2345 - auc: 0.9922
Epoch 89/100
1/1 [==============================] - 0s 31ms/step - loss: 0.2458 - auc: 0.9890
Epoch 90/100
1/1 [==============================] - 0s 26ms/step - loss: 0.2413 - auc: 0.9898
Epoch 91/100
1/1 [==============================] - 0s 26ms/step - loss: 0.2385 - auc: 0.9907
Epoch 92/100
1/1 [==============================] - 0s 27ms/step - loss: 0.2311 - auc: 0.9919
Epoch 93/100
1/1 [==============================] - 0s 24ms/step - loss: 0.2374 - auc: 0.9907
Epoch 94/100
1/1 [==============================] - 0s 25ms/step - loss: 0.2332 - auc: 0.9915
Epoch 95/100
1/1 [==============================] - 0s 24ms/step - loss: 0.2340 - auc: 0.9912
Epoch 96/100
1/1 [==============================] - 0s 35ms/step - loss: 0.2326 - auc: 0.9907
Epoch 97/100
1/1 [==============================] - 0s 25ms/step - loss: 0.2324 - auc: 0.9915
Epoch 98/100
1/1 [==============================] - 0s 26ms/step - loss: 

In [11]:
de=pd.DataFrame()
de['model']=model
de['auc']=AUC_ROC
de.to_csv('7feature_remove_one2.csv',index=False)
print(de)

               model       auc
0          all_model  0.949070
1  remove_AtomPairFP  0.927841
2      remove_Charge  0.948528
3       remove_Kappa  0.950307
4        remove_MAP4  0.946142
5         remove_MOE  0.949174
6    remove_MorganFP  0.948905
7    remove_Topology  0.951054
